# Track sample-level metadata

We already saw how to link data objects to entities representing features during ingestion.

For sample-level metadata, the underlying schema is often more complicated, and hence it's best done in a separate step.

Here, we walk through this process.

In [ ]:
import lamindb as ln
import lnschema_bionty as bt
import lnschema_lamin1 as ln1

ln.track()

Samples, i.e., metadata associated with observations, are linked with the same approach post-ingestion.

We'll need to lazily relationships of objects, and hence, we need to keep track of a session.

In [ ]:
ss = ln.Session()

Let's first query an scRNA-seq dataset stored as an `.h5ad` file.

In [ ]:
file = ss.select(ln.File, suffix=".h5ad").first()

In [ ]:
file

For instance, let's annotate a scRNA-seq dataset with its readout type (scRNA-seq), the tissue, and the species.

## Readout

In [ ]:
ro_lookup = bt.Readout.bionty.lookup()
scrnaseq = ro_lookup.single_cell_RNA_sequencing

scrnaseq

In [ ]:
readout = bt.Readout(name=scrnaseq.name)

readout

Link the readout against the data object.

In [ ]:
file.readouts.append(readout)

## Biosample

In [ ]:
biosample = ln1.Biosample(name="Mouse Lymph Node")

### Species

We already have mouse in the database, hence let's just query it. No need to create a new record.

In [ ]:
species = ln.select(bt.Species, name="mouse").one()

species

In [ ]:
biosample.species = species

### Tissue

In [ ]:
tissue_lookup = bt.Tissue.bionty.lookup()

In [ ]:
tissue_lookup.lymph_node

In [ ]:
tissue = bt.Tissue(name=tissue_lookup.lymph_node.name)

In [ ]:
tissue

In [ ]:
biosample.tissue = tissue

## Link against file

Link against the data object:

In [ ]:
file.biosamples.append(biosample)

## Add to the DB

We can add everything to the DB in one transaction:

In [ ]:
ss.add([readout, biosample])

Let us close the session.

In [ ]:
ss.close()

```{Tip}

Manage `Session` closing with a context manager instead of manually closing it!

With it the above would look like:

```{code}
with ln.Session() as ss:
    # manipulate data
```

## Query for linked metadata

In [ ]:
ln.select(ln.File).where(
    ln.File.readouts,
    bt.Readout.name == scrnaseq.name,
).df()

In [ ]:
ln.select(ln.File).join(ln.File.biosamples).where(
    ln1.Biosample.species, bt.Species.name == "mouse"
).df()

## What's in the database?

### Biological entities

In [ ]:
ln.view(schema="bionty")

### Wetlab

In [ ]:
ln.view(schema="lamin1")

In [ ]:
# integrity checks
with ln.Session() as ss:
    mouselymph = ss.select(ln.File, name="Mouse Lymph Node scRNA-seq").one()

    mouselymph_hash = mouselymph.hash
    assert mouselymph_hash == "Qprqj0O23197Ko-VobaZiw"

    mouselymph_features_hash = mouselymph.features[0].id
    assert mouselymph_features_hash == "2Mv3JtH-ScBVYHilbLaQ"